In [39]:
import pandas as pd
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple
from datetime import datetime

In [140]:
df = pd.read_csv("mpk_data.csv")
datetime_format = "%H:%M:%S"
df.head()

/var/folders/03/n179cc412h58x4qb9mt1bv_m0000gn/T/ipykernel_20016/2217797963.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("mpk_data.csv")


,Unnamed: 0,company,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
0,0,MPK Autobusy,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539
1,1,MPK Autobusy,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735
2,2,MPK Autobusy,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376
3,3,MPK Autobusy,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617
4,4,MPK Autobusy,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383


In [169]:
df_unique = df[[ 'start_stop' , 'start_stop_lat', 'start_stop_lon']].drop_duplicates()

df_unique['start_stop_index'] = df_unique.index + 1

df = df.drop(columns=['start_stop_index'], errors='ignore')

df = df.merge( df_unique[['start_stop' , 'start_stop_lat', 'start_stop_lon', 'start_stop_index']],
    on=['start_stop' , 'start_stop_lat', 'start_stop_lon'],
    how='left'
)


In [173]:
df.to_csv('mpk_indexed.csv')

In [134]:
@dataclass
class Connection:
    target_obj: 'BusStop'
    target_name: str
    line_name: str
    departure_t: datetime
    arrival_at: datetime

class BusStop:
    def __init__(self, latitude: float, longitude: float, 
                 name: str, connections: List[Connection] = None):
        self.name = name
        self.latitude = latitude
        self.longitude = longitude
        self.connections = connections if connections is not None else []

    def __str__(self):
        return f'{self.name} {self.latitude} {self.longitude}'

connection_graph: List[BusStop] = []

In [111]:
df_unique = df[['start_stop', 'start_stop_lat', 'start_stop_lon']].drop_duplicates()
df_unique.shape

(2264, 3)

In [135]:
for _, row in df_unique.iterrows():
    bus_stop = BusStop(
        name=row['start_stop'],
        latitude=row['start_stop_lat'],
        longitude=row['start_stop_lon'],
        connections=[]
                       )
    connection_graph.append(bus_stop)

In [120]:
df_unique[df_unique['start_stop'] == 'PL. GRUNWALDZKI']

,start_stop,start_stop_lat,start_stop_lon
14027,PL. GRUNWALDZKI,51.111621,17.060086
14161,PL. GRUNWALDZKI,51.111700,17.060432
53999,PL. GRUNWALDZKI,51.111141,17.061193
54015,PL. GRUNWALDZKI,51.111157,17.061465
64756,PL. GRUNWALDZKI,51.111623,17.059965
64786,PL. GRUNWALDZKI,51.111452,17.060529
97770,PL. GRUNWALDZKI,51.111444,17.059960
217287,PL. GRUNWALDZKI,51.111611,17.060434
543752,PL. GRUNWALDZKI,51.111438,17.060353
555984,PL. GRUNWALDZKI,51.111981,17.064034


In [ ]:
for x in [x for x in connection_graph if x.name == 'PL. GRUNWALDZKI']:
    print(x)

def get_bus_stop(name: str, lat: float, lon):
    bus_stop = [x for x in connection_graph 
                if x.name == name and x.latitude == lat and x.longitude == lon]
    return bus_stop

get_bus_stop(name='PL. GRUNWALDZKI',
             lat=51.11162051,
             lon=17.06008642)


PL. GRUNWALDZKI 51.11162051 17.06008642
PL. GRUNWALDZKI 51.11170028 17.06043151
PL. GRUNWALDZKI 51.11114106 17.0611933
PL. GRUNWALDZKI 51.1111569 17.06146483
PL. GRUNWALDZKI 51.11162272 17.05996513
PL. GRUNWALDZKI 51.11145165 17.06052919
PL. GRUNWALDZKI 51.11144435 17.05995961
PL. GRUNWALDZKI 51.11161063 17.0604342
PL. GRUNWALDZKI 51.11143827 17.0603531
PL. GRUNWALDZKI 51.11198112 17.06403361
PL. GRUNWALDZKI 51.11205745 17.06288628
PL. GRUNWALDZKI 51.111406 17.059852
PL. GRUNWALDZKI 51.11218871 17.06335752
PL. GRUNWALDZKI 51.11245891 17.0608282
PL. GRUNWALDZKI 51.11195744 17.06254965
PL. GRUNWALDZKI 51.11271353 17.0650486


[]